# Setup

In [71]:
import pandas as pd
import numpy as np
import itertools as it
pd.__version__, np.__version__

('0.19.2', '1.11.3')

In [72]:
# hack to make markdown code more visible.. (stolen from github css+)

In [73]:
%%html
<style type="text/css">
    .rendered_html code {
        padding: 0;
        padding-top: 0.2em;
        padding-bottom: 0.2em;
        margin: 0;
        font-size: 95%;
        background-color: rgba(27,31,35,0.05);
        border-radius: 3px;
        font-weight: bolder;
    }
</style>

stuff to look at:

- http://ehneilsen.net/notebook/pandasExamples/pandas_examples.html#orgheadline18

# Assignment with MI's

`idx = pd.IndexSlice` gives much more intuitive multi-index slicing

In [74]:
idx = pd.IndexSlice
midx = pd.MultiIndex.from_tuples([(1,'a'),(1,'c'),(2,'a'),(2,'b')])
mcols  = pd.MultiIndex.from_product([['ZZ','BB','QQ'],['oo','pp']])
s = pd.Series( list('quof'), index = midx)
print(s)

df = pd.DataFrame( np.random.random((4,6)) * 10 // 1 , index=midx, columns= mcols)
print(df)

df.loc[idx[:,'a'],['BB']] = df.loc[idx[:,'b'],['ZZ']].values

df

1  a    q
   c    u
2  a    o
   b    f
dtype: object
      ZZ        BB        QQ     
      oo   pp   oo   pp   oo   pp
1 a  7.0  7.0  9.0  9.0  3.0  4.0
  c  0.0  4.0  8.0  6.0  6.0  7.0
2 a  2.0  3.0  1.0  6.0  8.0  1.0
  b  2.0  2.0  8.0  7.0  2.0  4.0


ZZ        BB        QQ     
      oo   pp   oo   pp   oo   pp
1 a  7.0  7.0  2.0  2.0  3.0  4.0
  c  0.0  4.0  8.0  6.0  6.0  7.0
2 a  2.0  3.0  2.0  2.0  8.0  1.0
  b  2.0  2.0  8.0  7.0  2.0  4.0

In [75]:
idx = pd.IndexSlice
midx = pd.MultiIndex.from_tuples([(1,'a'),(1,'b'),(2,'a'),(2,'b')])
mcols  = pd.MultiIndex.from_product([['ZZ','BB','QQ'],['oo','pp']])
s = pd.Series( list('quof'), index = midx)


# df = pd.DataFrame( np.random.random((4,6)) * 10 // 1 , index=midx, columns= mcols)
print(df)

df.loc[idx[:,'a'],['BB']] = df.loc[idx[:,'b'],['ZZ']].values
df.loc[idx[:,'b'],['BB']] = 'hello!'
df.loc[idx[2,:],['BB','QQ']] = pd.to_timedelta(2, unit='s')

df

      ZZ        BB        QQ     
      oo   pp   oo   pp   oo   pp
1 a  7.0  7.0  2.0  2.0  3.0  4.0
  c  0.0  4.0  8.0  6.0  6.0  7.0
2 a  2.0  3.0  2.0  2.0  8.0  1.0
  b  2.0  2.0  8.0  7.0  2.0  4.0


ZZ                    BB                                QQ  \
      oo   pp               oo               pp               oo   
1 a  7.0  7.0                2                2                3   
  c  0.0  4.0                8                6                6   
2 a  2.0  3.0  0 days 00:00:02  0 days 00:00:02  0 days 00:00:02   
  b  2.0  2.0  0 days 00:00:02  0 days 00:00:02  0 days 00:00:02   

                      
                  pp  
1 a                4  
  c                7  
2 a  0 days 00:00:02  
  b  0 days 00:00:02

# Apply functions to different levels of columns

## using stack and unstack

In [76]:
idx = pd.IndexSlice
midx = pd.MultiIndex.from_tuples([(1,'a'),(1,'b'),(2,'a'),(2,'b')])
mcols  = pd.MultiIndex.from_product([['ZZ','BB','QQ'],['oo','pp']])
s = pd.Series( list('quof'), index = midx)


df = pd.DataFrame( np.random.random((4,6)) * 10 // 1 , index=midx, columns= mcols)
print(df)

df.loc[idx[:,'a'],['BB']] = df.loc[idx[:,'b'],['ZZ']].values

df.sortlevel(axis=1, inplace=True)
df

      ZZ        BB        QQ     
      oo   pp   oo   pp   oo   pp
1 a  4.0  8.0  3.0  7.0  0.0  0.0
  b  5.0  5.0  3.0  6.0  9.0  7.0
2 a  9.0  3.0  4.0  9.0  0.0  4.0
  b  2.0  3.0  9.0  1.0  7.0  3.0


BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  5.0  5.0  0.0  0.0  4.0  8.0
  b  3.0  6.0  9.0  7.0  5.0  5.0
2 a  2.0  3.0  0.0  4.0  9.0  3.0
  b  9.0  1.0  7.0  3.0  2.0  3.0

In the code below, we get the rank per column. How would I get the rank of a top-level column?
The only way I know is through stacking and unstacking.

In [77]:
df.apply(lambda c: c.rank())

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  3.0  3.0  1.5  1.0  2.0  4.0
  b  2.0  4.0  4.0  4.0  3.0  3.0
2 a  1.0  2.0  1.5  3.0  4.0  1.5
  b  4.0  1.0  3.0  2.0  1.0  1.5

In [78]:
df.stack().apply(lambda c: c.rank()).unstack()#.swaplevel(1,0, axis=1).sortlevel(axis=1)

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  5.5  5.5  2.0  2.0  4.0  7.0
  b  3.5  7.0  8.0  6.5  5.5  5.5
2 a  2.0  3.5  2.0  5.0  8.0  2.5
  b  8.0  1.0  6.5  4.0  1.0  2.5

## Approach 2: use groupby (__ it has a level parameter __)

this won't work because it does grouping...

In [79]:
df

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  5.0  5.0  0.0  0.0  4.0  8.0
  b  3.0  6.0  9.0  7.0  5.0  5.0
2 a  2.0  3.0  0.0  4.0  9.0  3.0
  b  9.0  1.0  7.0  3.0  2.0  3.0

OMG look at this - I rank the rows

In [80]:
df.T.groupby(level=1).rank().T

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  3.0  2.0  1.0  1.0  2.0  3.0
  b  1.0  2.0  3.0  3.0  2.0  1.0
2 a  2.0  1.5  1.0  3.0  3.0  1.5
  b  3.0  1.0  2.0  2.5  1.0  2.5

In [81]:
df.groupby(level=1,axis=1).rank()

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  3.0  2.0  1.0  1.0  2.0  3.0
  b  1.0  2.0  3.0  3.0  2.0  1.0
2 a  2.0  1.5  1.0  3.0  3.0  1.5
  b  3.0  1.0  2.0  2.5  1.0  2.5

In [82]:
#only this ranks the rows...
df.apply(lambda r: r.rank(),axis=1)

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  4.5  4.5  1.5  1.5  3.0  6.0
  b  1.0  4.0  6.0  5.0  2.5  2.5
2 a  2.0  3.5  1.0  5.0  6.0  3.5
  b  6.0  1.0  5.0  3.5  2.0  3.5

In [83]:
df.groupby(level=1,axis=0).rank()

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  2.0  2.0  1.5  1.0  1.0  2.0
  b  1.0  2.0  2.0  2.0  2.0  2.0
2 a  1.0  1.0  1.5  2.0  2.0  1.0
  b  2.0  1.0  1.0  1.0  1.0  1.0

In [84]:
df.groupby(level=0,axis=0).rank()

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  2.0  1.0  1.0  1.0  1.0  2.0
  b  1.0  2.0  2.0  2.0  2.0  1.0
2 a  1.0  2.0  1.0  2.0  2.0  1.5
  b  2.0  1.0  2.0  1.0  1.0  1.5

## how's about using indexing, stacking, and unstacking etc

but how is this different than just stacking and unstacking?

could I make a generic function that takes a given level, and applies a func to it a returns something of a similar shape?

need to better understand `reindex` and `align` methods which now take level functions

# Using reindex and align



In [85]:
df

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  5.0  5.0  0.0  0.0  4.0  8.0
  b  3.0  6.0  9.0  7.0  5.0  5.0
2 a  2.0  3.0  0.0  4.0  9.0  3.0
  b  9.0  1.0  7.0  3.0  2.0  3.0

In [86]:
df1 = df.copy()
df1.index = df.index.droplevel()
df1

BB        QQ        ZZ     
    oo   pp   oo   pp   oo   pp
a  5.0  5.0  0.0  0.0  4.0  8.0
b  3.0  6.0  9.0  7.0  5.0  5.0
a  2.0  3.0  0.0  4.0  9.0  3.0
b  9.0  1.0  7.0  3.0  2.0  3.0

In [87]:
df2 = df.loc[1,:]
df2

BB        QQ        ZZ     
    oo   pp   oo   pp   oo   pp
a  5.0  5.0  0.0  0.0  4.0  8.0
b  3.0  6.0  9.0  7.0  5.0  5.0

## So to reindex to finer granularity we can use `reindex` and the `level` parameter


The `level` parameter tells which level of the reindexing dataframe to use to bind with the dataframe that will be reindexed.

In [88]:

df2.reindex(df.index, level=1)

BB        QQ        ZZ     
      oo   pp   oo   pp   oo   pp
1 a  5.0  5.0  0.0  0.0  4.0  8.0
  b  3.0  6.0  9.0  7.0  5.0  5.0
2 a  5.0  5.0  0.0  0.0  4.0  8.0
  b  3.0  6.0  9.0  7.0  5.0  5.0

## MI algebra

### break mi into tuples 

In [89]:
midx = pd.MultiIndex.from_tuples([(1,'a'),(1,'c'),(2,'a'),(2,'b')])

In [90]:
[midx.get_level_values(i) for i in range(midx.nlevels)]

[Int64Index([1, 1, 2, 2], dtype='int64'),
 Index(['a', 'c', 'a', 'b'], dtype='object')]

In [91]:
zl = list(zip(*[midx.get_level_values(i) for i in range(midx.nlevels)]))
zl

[(1, 'a'), (1, 'c'), (2, 'a'), (2, 'b')]

In [92]:
pd.MultiIndex.from_tuples(zl)

MultiIndex(levels=[[1, 2], ['a', 'b', 'c']],
           labels=[[0, 0, 1, 1], [0, 2, 0, 1]])

### Create play indices

In [93]:
ndx_d = midx

** NB.** `mi.values` returns an array of tuples 

In [94]:
ndx_d.values #hilarious.. == list(zip(*[midx.get_level_values(i) for i in range(midx.nlevels)]))

array([(1, 'a'), (1, 'c'), (2, 'a'), (2, 'b')], dtype=object)

In [95]:
ndx_a = pd.MultiIndex.from_product( [['BB', 'QQ', 'ZZ'],['ss', 'tt']])

ndx_b = pd.MultiIndex.from_product([['AA', 'CC', 'ZZ'],['qq', '33']])

In [96]:
ndx_a

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['ss', 'tt']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [97]:
ndx_b

MultiIndex(levels=[['AA', 'CC', 'ZZ'], ['33', 'qq']],
           labels=[[0, 0, 1, 1, 2, 2], [1, 0, 1, 0, 1, 0]])

### union

Looking below, we get a surprise: whereas we might have expected to have the same number of elements for each ['AA', 'BB', 'CC', 'QQ', 'ZZ'], we see that this
is not the case. While the `levels` are unions, the `labels` are a result of taking the union of the tuples of the parameters.

In [98]:

_1 = pd.DataFrame(np.arange(6).reshape((1,6)), columns=ndx_a)
_1

BB    QQ    ZZ   
  ss tt ss tt ss tt
0  0  1  2  3  4  5

In [99]:

_1 = pd.DataFrame(np.arange(6*6).reshape((6,6)), index=ndx_a, columns=ndx_b)
_1

AA      CC      ZZ    
       qq  33  qq  33  qq  33
BB ss   0   1   2   3   4   5
   tt   6   7   8   9  10  11
QQ ss  12  13  14  15  16  17
   tt  18  19  20  21  22  23
ZZ ss  24  25  26  27  28  29
   tt  30  31  32  33  34  35

We make a union of the two indices, note that `ZZ` is in both indices.

In [100]:
ndx_u = ndx_a.union(ndx_b)
ndx_u

MultiIndex(levels=[['AA', 'BB', 'CC', 'QQ', 'ZZ'], ['33', 'qq', 'ss', 'tt']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]],
           sortorder=0)

In [101]:

_df = pd.DataFrame(np.arange(6*6).reshape((12,3)), index=ndx_u)
_df

0   1   2
AA 33   0   1   2
   qq   3   4   5
BB ss   6   7   8
   tt   9  10  11
CC 33  12  13  14
   qq  15  16  17
QQ ss  18  19  20
   tt  21  22  23
ZZ 33  24  25  26
   qq  27  28  29
   ss  30  31  32
   tt  33  34  35

In [102]:
ndx_a

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['ss', 'tt']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [103]:
pd.MultiIndex.from_arrays([ndx_a.get_level_values(0), ndx_a.get_level_values(1)])

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['ss', 'tt']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [104]:
list(zip(*[ndx_a.get_level_values(0), ndx_a.get_level_values(1)]))

[('BB', 'ss'),
 ('BB', 'tt'),
 ('QQ', 'ss'),
 ('QQ', 'tt'),
 ('ZZ', 'ss'),
 ('ZZ', 'tt')]

## append_to_level?

eg. in `df2` below I would like to add `gg` to `oo` and `pp`

In [208]:
def append_to_level(ndx, el, level=-1):
    """ ndx: is a pandas Multi-index
    el is a label (str or int)
    level is which of multi-index to append to
    returns a full tree emulti-index in which a level has an extra element appended to it 
    """
    a = pd.DataFrame.from_records(ndx.tolist())#.values.tolist())
    ac = a.copy()
    ac.iloc[:,level] = el
    aac = a.append(ac).drop_duplicates()
    tup_l = [tuple(r) for i,r in aac.iterrows()]
    return pd.MultiIndex.from_tuples(tup_l)
# ndx.values

### Example usage


In [216]:
mi = pd.MultiIndex.from_product([['a','b','c'],['W','X','Y'],['0','1']],names=['mini','magi','bin'] )

In [239]:
_df = pd.DataFrame(np.arange(2*18).reshape((18,2)), index=mi)
_df

0   1
mini magi bin        
a    W    0     0   1
          1     2   3
     X    0     4   5
          1     6   7
     Y    0     8   9
          1    10  11
b    W    0    12  13
          1    14  15
     X    0    16  17
          1    18  19
     Y    0    20  21
          1    22  23
c    W    0    24  25
          1    26  27
     X    0    28  29
          1    30  31
     Y    0    32  33
          1    34  35

In [240]:
# we shove _df into the new index
new_mi = append_to_level(_df.index,'V',1)
_df2 = _df.reindex(index=new_mi).sortlevel(axis=0)
_df2

0     1
a V 0   NaN   NaN
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   4.0   5.0
    1   6.0   7.0
  Y 0   8.0   9.0
    1  10.0  11.0
b V 0   NaN   NaN
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  16.0  17.0
    1  18.0  19.0
  Y 0  20.0  21.0
    1  22.0  23.0
c V 0   NaN   NaN
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  28.0  29.0
    1  30.0  31.0
  Y 0  32.0  33.0
    1  34.0  35.0

In [241]:
# this works because our assignment is perpendicular to the change in index
# what?? I didn't change the index of things
_df3 = _df2.copy()
_df3.loc[idx[:,['X','Y'],'0'],0] = _df3.loc[idx[:,['X','Y'],'0'],1]

_df3

0     1
a V 0   NaN   NaN
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   5.0   5.0
    1   6.0   7.0
  Y 0   9.0   9.0
    1  10.0  11.0
b V 0   NaN   NaN
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  17.0  17.0
    1  18.0  19.0
  Y 0  21.0  21.0
    1  22.0  23.0
c V 0   NaN   NaN
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  29.0  29.0
    1  30.0  31.0
  Y 0  33.0  33.0
    1  34.0  35.0

In [242]:
# this should not work...
_df4 = _df2.copy()
_df4.loc[idx[:,['X','Y'],'0'],0] = _df4.loc[idx[:,['X','Y'],'1'],0]
_df4

# and it doesn't because the assignment is looking for the right index and it can't find it. but if I remove the index, it should work...

0     1
a V 0   NaN   NaN
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   NaN   5.0
    1   6.0   7.0
  Y 0   NaN   9.0
    1  10.0  11.0
b V 0   NaN   NaN
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0   NaN  17.0
    1  18.0  19.0
  Y 0   NaN  21.0
    1  22.0  23.0
c V 0   NaN   NaN
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0   NaN  29.0
    1  30.0  31.0
  Y 0   NaN  33.0
    1  34.0  35.0

In [243]:
# this should not work...
_df5 = _df2.copy()
_df5.loc[idx[:,['X','Y'],'0'],0] = _df5.loc[idx[:,['X','Y'],'1'],0].values
_df5

# and it doesn't because the assignment is looking for the right index and it can't find it. but if I remove the index, it should work...
# and it does! yessss...

0     1
a V 0   NaN   NaN
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   6.0   5.0
    1   6.0   7.0
  Y 0  10.0   9.0
    1  10.0  11.0
b V 0   NaN   NaN
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  18.0  17.0
    1  18.0  19.0
  Y 0  22.0  21.0
    1  22.0  23.0
c V 0   NaN   NaN
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  30.0  29.0
    1  30.0  31.0
  Y 0  34.0  33.0
    1  34.0  35.0

In [248]:
# and this works too, and is worth studying!
# using sum with a level parameter means that all levels EXCEPT the given level
# will be summed out.

_df6 = _df2.copy()
print(_df6.loc[idx[:,'V','0'],0].shape) 
print(_df6.loc[idx[:,['W','X','Y'],'0'],0].sum(level=0,axis=0))
_df6.loc[idx[:,'V','0'],0] = _df6.loc[idx[:,['W','X','Y'],'0'],0].sum(level=0,axis=0).values
_df6


(3,)
a    12.0
b    48.0
c    84.0
Name: 0, dtype: float64


0     1
a V 0  12.0   NaN
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   4.0   5.0
    1   6.0   7.0
  Y 0   8.0   9.0
    1  10.0  11.0
b V 0  48.0   NaN
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  16.0  17.0
    1  18.0  19.0
  Y 0  20.0  21.0
    1  22.0  23.0
c V 0  84.0   NaN
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  28.0  29.0
    1  30.0  31.0
  Y 0  32.0  33.0
    1  34.0  35.0

In [252]:
# and this works too, and is worth studying!
# using sum with a level parameter means that all levels EXCEPT the given level
# will be summed out.

_df7 = _df2.copy()
print(_df7.loc[idx[:,'V','0'],:].shape) 
print(_df7.loc[idx[:,['W','X','Y'],'0'],:].sum(level=0,axis=0))
_df7.loc[idx[:,'V','0'],:] = _df7.loc[idx[:,['W','X','Y'],'0'],:].sum(level=0,axis=0).values
_df7


(3, 2)
        0     1
a W   0.0   1.0
  X   4.0   5.0
  Y   8.0   9.0
b W  12.0  13.0
  X  16.0  17.0
  Y  20.0  21.0
c W  24.0  25.0
  X  28.0  29.0
  Y  32.0  33.0


0     1
a V 0  12.0  15.0
    1   NaN   NaN
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   4.0   5.0
    1   6.0   7.0
  Y 0   8.0   9.0
    1  10.0  11.0
b V 0  48.0  51.0
    1   NaN   NaN
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  16.0  17.0
    1  18.0  19.0
  Y 0  20.0  21.0
    1  22.0  23.0
c V 0  84.0  87.0
    1   NaN   NaN
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  28.0  29.0
    1  30.0  31.0
  Y 0  32.0  33.0
    1  34.0  35.0

In [254]:
# and this is where shit gets crazy
# the levels parameter in sum can take a list

# I want to keep 'a' _ '0' and sum out the capitals ['W','X','Y']
# .... and see below. very cool.

_df7 = _df2.copy()
print(_df7.loc[idx[:,'V','0'],:].shape) 
print(_df7.loc[idx[:,['W','X','Y'],:],:].sum(level=[0,2],axis=0))
_df7.loc[idx[:,'V',:],:] = _df7.loc[idx[:,['W','X','Y'],:],:].sum(level=[0,2],axis=0).values
_df7


(3, 2)
        0     1
a 0  12.0  15.0
  1  18.0  21.0
b 0  48.0  51.0
  1  54.0  57.0
c 0  84.0  87.0
  1  90.0  93.0


0     1
a V 0  12.0  15.0
    1  18.0  21.0
  W 0   0.0   1.0
    1   2.0   3.0
  X 0   4.0   5.0
    1   6.0   7.0
  Y 0   8.0   9.0
    1  10.0  11.0
b V 0  48.0  51.0
    1  54.0  57.0
  W 0  12.0  13.0
    1  14.0  15.0
  X 0  16.0  17.0
    1  18.0  19.0
  Y 0  20.0  21.0
    1  22.0  23.0
c V 0  84.0  87.0
    1  90.0  93.0
  W 0  24.0  25.0
    1  26.0  27.0
  X 0  28.0  29.0
    1  30.0  31.0
  Y 0  32.0  33.0
    1  34.0  35.0

### other version of append_to_level

In [106]:
def append_to_level0(ndx, el, level=-1):
    """ ndx: is a pandas Multi-index
    el is a label (str or int)
    level is which of multi-index to append to
    returns a full tree emulti-index in which a level has an extra element appended to it """
    _l = [ l.tolist() for l in ndx.levels]
    _l[level].append(el)
    # apply sortlevel appends a sorting array to our index which makes other functions upset..
    # looks buggish
    return pd.MultiIndex.from_product(_l)#.sortlevel()

In [111]:
def append_to_level2(ndx, el, level=-1):
    """
    given a list of tuples of equal length, we will
    let k_i be the number of values in level i,
    we then create \prod(i <> 'level') k_i number of tuples 
    which have the value 'el' in position 'level'
    
    this function isn't necessary...
    
    it would apply when a tree is not a cartesian product
    """
    # simply copy every tuple in ndx and replace actual level value
    # 'el'. we then take the unique values of this list and append it
    # to original list.
    tpl = []
    for t in ndx.values:
        tl = list(t)
        tl[level] = el
        tpl.append(tuple(tl))

    new_tuples = ndx.values.tolist() + list(set(tpl))
    _idx = pd.MultiIndex.from_tuples(new_tuples)
    return _idx

### Union

In [114]:
df2

BB        QQ        ZZ     
    oo   pp   oo   pp   oo   pp
a  5.0  5.0  0.0  0.0  4.0  8.0
b  3.0  6.0  9.0  7.0  5.0  5.0

In [207]:
df2.columns.values

array([('BB', 'oo'), ('BB', 'pp'), ('QQ', 'oo'), ('QQ', 'pp'),
       ('ZZ', 'oo'), ('ZZ', 'pp')], dtype=object)

In [115]:
df2.columns.tolist()

[('BB', 'oo'),
 ('BB', 'pp'),
 ('QQ', 'oo'),
 ('QQ', 'pp'),
 ('ZZ', 'oo'),
 ('ZZ', 'pp')]

In [116]:
df2.columns

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['oo', 'pp']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Like this:

In [117]:
ndx = df.columns
ndx2 = pd.MultiIndex.from_tuples(ndx.tolist() + [(
    u, b) for u in ['BB', 'QQ', 'ZZ'] for b in ['ss', 'tt']])
df3 = df.reindex(columns=ndx2).sortlevel(axis=1)
df3

BB                QQ                ZZ             
      oo   pp  ss  tt   oo   pp  ss  tt   oo   pp  ss  tt
1 a  5.0  5.0 NaN NaN  0.0  0.0 NaN NaN  4.0  8.0 NaN NaN
  b  3.0  6.0 NaN NaN  9.0  7.0 NaN NaN  5.0  5.0 NaN NaN
2 a  2.0  3.0 NaN NaN  0.0  4.0 NaN NaN  9.0  3.0 NaN NaN
  b  9.0  1.0 NaN NaN  7.0  3.0 NaN NaN  2.0  3.0 NaN NaN

alternative solution: get product of first idx with another the multiplier
I'm assuming I can multiply indexes ...

possible tools `union` and `intersection` (need acceeds to levels...)

MI.append() may do what we want..

In [118]:
df3.columns

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['oo', 'pp', 'ss', 'tt']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

In [119]:
df3cp = df3.columns.append(df3.columns)
df3cp

MultiIndex(levels=[['BB', 'QQ', 'ZZ'], ['oo', 'pp', 'ss', 'tt']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]])

In [120]:
df.reindex(columns=df3cp).T

1         2     
         a    b    a    b
BB oo  5.0  3.0  2.0  9.0
   pp  5.0  6.0  3.0  1.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN
QQ oo  0.0  9.0  0.0  7.0
   pp  0.0  7.0  4.0  3.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN
ZZ oo  4.0  5.0  9.0  2.0
   pp  8.0  5.0  3.0  3.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN
BB oo  5.0  3.0  2.0  9.0
   pp  5.0  6.0  3.0  1.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN
QQ oo  0.0  9.0  0.0  7.0
   pp  0.0  7.0  4.0  3.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN
ZZ oo  4.0  5.0  9.0  2.0
   pp  8.0  5.0  3.0  3.0
   ss  NaN  NaN  NaN  NaN
   tt  NaN  NaN  NaN  NaN

must get levels !

nomenclature: does `levels` refer the different levels in the hierachy of a muli-index _or_ does it refer to the categories within. 

In [121]:
def level_lists(ndx, lvls=[]):
    return [ndx.levels[level].tolist() for level in lvls]


In [122]:
level_lists(ndx,[0,1])

[['BB', 'QQ', 'ZZ'], ['oo', 'pp']]

In [123]:
pd.MultiIndex.from_product( level_lists(ndx,[0]))

Index(['BB', 'QQ', 'ZZ'], dtype='object')

`mi.append` basically gets concatenates two list of tuples

In [124]:
__idx = ndx.append(pd.MultiIndex.from_product( level_lists(ndx,[1,0])))
__idx

MultiIndex(levels=[['BB', 'QQ', 'ZZ', 'oo', 'pp'], ['BB', 'QQ', 'ZZ', 'oo', 'pp']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4], [3, 4, 3, 4, 3, 4, 0, 1, 2, 0, 1, 2]])

self observation: I will tend to abstraction over getting shit done...

In [125]:
df.reindex(__idx)

BB      QQ      ZZ    
       oo  pp  oo  pp  oo  pp
BB oo NaN NaN NaN NaN NaN NaN
   pp NaN NaN NaN NaN NaN NaN
QQ oo NaN NaN NaN NaN NaN NaN
   pp NaN NaN NaN NaN NaN NaN
ZZ oo NaN NaN NaN NaN NaN NaN
   pp NaN NaN NaN NaN NaN NaN
oo BB NaN NaN NaN NaN NaN NaN
   QQ NaN NaN NaN NaN NaN NaN
   ZZ NaN NaN NaN NaN NaN NaN
pp BB NaN NaN NaN NaN NaN NaN
   QQ NaN NaN NaN NaN NaN NaN
   ZZ NaN NaN NaN NaN NaN NaN

In [145]:
def append_to_level4( item_l, axis = 1, level = -1):
    ndx = df.columns
    ndx2 = pd.MultiIndex.from_tuples(ndx.tolist() + [(
        u, b) for u in ['BB', 'QQ', 'ZZ'] for b in ['ss', 'tt']])
    df3 = df.reindex(columns=ndx2).sortlevel(axis=1)
    df3

In [146]:
ndx.levshape

(3, 2)

In [147]:
ndx.levels

FrozenList([['BB', 'QQ', 'ZZ'], ['oo', 'pp']])

In [148]:
def addlevel(mi, level_values, level=-1):
    # how many levels
    shape = mi.levshape
    
    # sort levels
    

## assiging sub frames is ugly

Currently, I have to pull data out with `values`. Aren't the indexes/columns ligning up?. 

Can we fix this with `align` ?

In [149]:
# why must this be so ugly?
df3.loc[:,idx[:,'ss']] = df3.loc[:,idx[:,'oo']].values + df3.loc[:,idx[:,'pp']].values
df3.loc[:,idx[:,'tt']] = df3.loc[:,idx[:,'oo']].values * df3.loc[:,idx[:,'pp']].values


In [150]:
df3

BB                    QQ                    ZZ                 
      oo   pp    ss    tt   oo   pp    ss    tt   oo   pp    ss    tt
1 a  5.0  5.0  10.0  25.0  0.0  0.0   0.0   0.0  4.0  8.0  12.0  32.0
  b  3.0  6.0   9.0  18.0  9.0  7.0  16.0  63.0  5.0  5.0  10.0  25.0
2 a  2.0  3.0   5.0   6.0  0.0  4.0   4.0   0.0  9.0  3.0  12.0  27.0
  b  9.0  1.0  10.0   9.0  7.0  3.0  10.0  21.0  2.0  3.0   5.0   6.0

# Experiments with `align`

In [151]:
_s = pd.Series(range(4),df.index); _s

1  a    0
   b    1
2  a    2
   b    3
dtype: int64

In [152]:
s_index = _s.index.set_names(['num','lower'])

In [153]:
_s.index = s_index

In [154]:
_s.index

MultiIndex(levels=[[1, 2], ['a', 'b']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['num', 'lower'])

In [159]:
ndx.levels[0]

Index(['BB', 'QQ', 'ZZ'], dtype='object')

In [160]:
ndx.to_series()

BB  oo    (BB, oo)
    pp    (BB, pp)
QQ  oo    (QQ, oo)
    pp    (QQ, pp)
ZZ  oo    (ZZ, oo)
    pp    (ZZ, pp)
dtype: object

In [161]:
ndx.tolist()

[('BB', 'oo'),
 ('BB', 'pp'),
 ('QQ', 'oo'),
 ('QQ', 'pp'),
 ('ZZ', 'oo'),
 ('ZZ', 'pp')]

In [162]:
# must convert idx.levels into list of lists to make work... (a github pull request?)
ndx2 = pd.MultiIndex.from_product( [list(l) for l in _s.index.levels] + [['A','T']]); ndx2

MultiIndex(levels=[[1, 2], ['a', 'b'], ['A', 'T']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1]])

In [163]:
def MIproduct(mi, iterable):
    return pd.MultiIndex.from_product( [list(l) for l in _s.index.levels] + [list(l) for l in iterable]); ndx2

In [164]:
ndx3 = MIproduct(ndx, [['A','T']]).set_names(['num','lower','upper']); ndx3

MultiIndex(levels=[[1, 2], ['a', 'b'], ['A', 'T']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['num', 'lower', 'upper'])

In [165]:
_s2 = pd.Series(range(8), MIproduct(_s.index,[['A','T']])); _s2

1  a  A    0
      T    1
   b  A    2
      T    3
2  a  A    4
      T    5
   b  A    6
      T    7
dtype: int64

# Splitting columns and rows

